In [18]:
#importing relevant packages

import numpy as np
import time
import sys
import ssl
import json
from ast import literal_eval
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from elasticsearch.connection import create_ssl_context
import datetime

In [1]:
# !pip install elasticsearch

In [16]:
# creating context instance 

context = create_ssl_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

In [17]:
# Instantiating elastic search with credentials

es = Elasticsearch(
    '****',
    http_auth=('***', '******'),
    scheme="https",
    port=9200,
    ssl_context = context,
)

In [19]:
# pre-processing for columns from string to their respective formats

df=pd.read_csv("ted_main_preprocessing (2).csv")
#converting string to json
df["ratings"]=df["ratings"].apply(lambda x:json.loads(x))
#converting string to list
df["tags"]=df["tags"].apply(literal_eval)
df["speaker_occupation"]=df["speaker_occupation"].apply(literal_eval)
#converting epoch to datetime
df["published_date"]=df["published_date"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
df["film_date"]=df["film_date"].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d'))
df.head(5)

,comments,description,duration,event,film_date,main_speaker,num_speaker,published_date,ratings,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,2006-02-24,Ken Robinson,1,2006-06-26,"{'Funny': 19645, 'Beautiful': 4573, 'Ingenious...","[Author, educator]","[children, creativity, culture, dance, educati...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,2006-02-24,Al Gore,1,2006-06-26,"{'Funny': 544, 'Courageous': 139, 'Confusing':...",[Climate advocate],"[alternative energy, cars, climate change, cul...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,2006-02-23,David Pogue,1,2006-06-26,"{'Funny': 964, 'Courageous': 45, 'Ingenious': ...",[Technology columnist],"[computers, entertainment, interface design, m...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,2006-02-25,Majora Carter,1,2006-06-26,"{'Courageous': 760, 'Beautiful': 291, 'Confusi...",[Activist for environmental justice],"[MacArthur grant, activism, business, cities, ...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,2006-02-21,Hans Rosling,1,2006-06-27,"{'Ingenious': 3202, 'Funny': 1390, 'Beautiful'...","[Global health expert, data visionary]","[Africa, Asia, Google, demo, economics, global...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [20]:
# extracting tags to display on UI
tags_list=[]
for i in df["tags"]:
    tags_list.extend(i)

In [22]:
# code to load the data into respective index
itera = 0
count = 0
ACTIONS = []
for ind in df.index: 

    a,b,c,d,e,f,g,h,i,j,k,l,m,n=df['comments'][ind],df['description'][ind],df['duration'][ind],df['event'][ind],df['film_date'][ind],df['main_speaker'][ind],df['num_speaker'][ind],df['published_date'][ind],df['ratings'][ind],df['speaker_occupation'][ind],df['tags'][ind],df['title'][ind],df['url'][ind],df['views'][ind]

    action = {
        "_index": "info624_201904_final_project",
        "_source": {
              "comments": a,
              "description": b, 
              "duration":c,
              "event":d,
              "film_date":e,
              "main_speaker":f,
              "num_speaker":g,
              "published_date":h,
              "ratings":i,
              "speaker_occupation":j,
              'tags':k,
              "title":l,
              "url":m,
              "views":n,
        },
        "_id":ind
        
    }
    itera += 1
    ACTIONS.append(action)


success, _ = bulk(es, ACTIONS, index = "info624_201904_final_project", raise_on_error=True)



C:\Users\likhi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\likhi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\likhi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\likhi\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTT